In [1]:
import requests

In [25]:
# requsets.request는 웹서버에 request를 요청하는 메서드이다.
# 첫번째 파라미터는 웹통신 방식으로서, "get" 또는 "post" 중 하나이다. -> 개발자 도구/네트워크/헤더 정보에서 확인할 수 있다.
# 두번째 파라미터는 url 주소이다. 
# shift + tab 키를 해보면 파라미터가 더 있으며 그것들은 옵션이다. 

# 아래 코드에서 사용한 사이트는 status code 에러 연습용 사이트이며, url 주소 끝에 에러번호에 따라서 해당 에러를 발생시킨다.
# 500번대 에러를 발생시켜보자.

resp = requests.request("get", "https://www.crawler-test.com/status_codes/status_505")
resp

<Response [505]>

In [26]:
# status_code는 에러코드를 반환한다.

resp.status_code

505

In [27]:
# reason은 에러코드에 따른 설명을 반환한다.

resp.reason

'HTTP Version Not Supported'

In [28]:
# 에러코드가 뜨면 에러를 발생시키는 메서드.
# requsets 패키지는 response를 정상적으로 받아오지 않더라도 커널에서 에러를 발생시키지 않는다.
# 위에서 response가 505 에러를 받아왔지만, 커널에서 에러는 나지 않았었다.
# request를 날린 것 만으로는 커널에서 에러가 나지 않는다는 점을 항상 유의할 것.

resp.raise_for_status()

HTTPError: 505 Server Error: HTTP Version Not Supported for url: https://www.crawler-test.com/status_codes/status_505

In [20]:
# 정상 통신했을때는 어떤지 보자

resp = requests.request("get", "https://www.crawler-test.com/status_codes/status_200")
resp

<Response [200]>

In [21]:
# status_code 200은 정상이라는 뜻이다

resp.status_code

200

In [22]:
# 에러 이유는 없으므로 OK

resp.reason

'OK'

In [24]:
# status가 정상이므로 에러를 발생시키지 않는다.

resp.raise_for_status()

---

# 나만의 download 함수 만들기

In [35]:
# time 패키지를 함수에서 활용할 것이므로 미리 import.
import time

# 예외처리문을 활용해서 나만의 함수를 만들어보자.
# HTTPError가 발생하면 알려주거나, 알아서 해결하게 만들어보자.

def download(method, url, params=None, data=None, headers=None, timeout=1, retries=4):
    # User-Agent 정보가 없으면 response 결과가 다른 경우가 있으므로 기본값을 넣어주자. (예를 들어 google, naver 등)
    # 자신의 헤더정보는 브라우저 개발자도구에서 확인할 수 있다.
    if headers == None:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
    
    # 예외처리문
    try:
        resp = requests.request(method, url, 
                                params=params, data=data, 
                                timeout=timeout)
        resp.raise_for_status() # 정상적으로 response를 못 받아오면 Error를 발생시킨다. -> except 문 실행.
        
    # 500번대 에러는 서버 에러로서, 서버에서 request를 모두 처리하지 못 해서 발생하는 경우가 많다. 
    # 500번대 에러는 시간을 좀 두고 다시 request를 날리면 해결되는 경우가 많다.
    # 무한 재귀에 빠질 위험이 있으므로 재시도 횟수를 조건에 포함시켜주자.
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print("remained retry:", retries)
            time.sleep(timeout)
            resp = download(method, url, params, data, headers, timeout, retries-1) # retry 횟수를 하나 삭감해서 전달한다.
        else:
            print(resp.status_code)
            print(resp.reason)
        
    return resp

In [36]:
# 다운로드 함수를 써보자.

resp = download('get', "https://www.google.com")

In [37]:
# download 함수가 return한 값 자체는 requests.request가 return한 값과 동일하다.

resp

<Response [200]>

In [38]:
# .text 메소드는 request에 대하여 서버가 응답하여 보내온 html 코드를 보여준다.

resp.text

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="jAqLLAXwm74ZOnGFJrHz7g==">(function(){window.google={kEI:\'y_4pXdjBFMel8AXnyabYAg\',kEXPI:\'0,1353804,1957,1642,781,1225,730,224,510,1065,3152,377,207,1017,54,122,51,541,295,90,54,75,201,54,187,55,318,90,10,71,159,1133430,1197778,329489,1294,12383,4855,32692,15247,867,12163,16521,363,3320,5505,2442,260,5107,575,835,284,2,579,727,2432,1361,4323,4968,773,2247,7865,2595,2287,1314,669,1050,1808,1397,81,7,491,2044,8604,305,1908,2696,692,797,101,1119,38,920,746,8,119,1217,1364,346,1,1264,2736,3061,2,631,3248,36,1472,2,2674,635,1160,1447,632,2228,656,20,318,1117,448,454,104,1247,777,1,369,1316,705,756,98,392,30,399,471,521,1107,10,168,8,109,1018,1495,174,889,78,48,553,11,14,10,1269,2212,25,177,323,5,1245,7,840,32

In [39]:
# 어떤 url로부터 response 받아왔는지 알려준다.

resp.url

'https://www.google.com/'

In [40]:
# response에 대한 header 정보를 보여준다.

resp.headers

{'Date': 'Sat, 13 Jul 2019 15:54:51 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=ISO-8859-1', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2019-07-13-15; expires=Mon, 12-Aug-2019 15:54:51 GMT; path=/; domain=.google.com, NID=187=PZWqesN4E6fYL8bFpoJrESJdZkb8KTJIH7jwLcxAXhwXuIT8XM6CAJSLOewYJfbzWLt8QFoTNmcYZymZfUkQ7hJx29oVSN39LJAn15J9uUOHBgfwU5Z6hB3GGSmhoRmT4XuvPBHBmKMGFU_2rFcltppeXq1s5FKmWEtQhNqZshY; expires=Sun, 12-Jan-2020 15:54:51 GMT; path=/; domain=.google.com; HttpOnly', 'Alt-Svc': 'quic=":443"; ma=2592000; v="46,43,39"', 'Transfer-Encoding': 'chunked'}

---